# PLAYGROUND for AgentGENius

In [1]:
from datetime import date

import nest_asyncio
from dotenv import load_dotenv
from pydantic_ai import RunContext, Tool

from agentgenius.agents import AgentStore, BaseAgent
from agentgenius.config import config
from agentgenius.main import AgentGENius
from agentgenius.tools import ToolSet

nest_asyncio.apply()
load_dotenv()

True

## Testing AgentStore 

In [2]:
ags = AgentStore(config.agents_path).load_agents()
# ags.add(AgentGENius.from_file(config.agents_path / "assistant.json"))
a = ags.list()[0]
print(a)
if agent := ags[a]:
    print(agent.run_sync("write hello world").data)
    print(agent.toolset)

ROT13_encoder
In ROT13, "hello world" is encoded as "uryyb jbeyq".
ToolSet(['rot13_encode', 'ask_agent'])


## Using tools

In [3]:
def encode_rot13(input_string):
    """Encode a string using ROT13."""
    import codecs

    return codecs.encode(input_string, "rot_13")


# check signature if function is plain
# for DEV purposes only
a = encode_rot13
import inspect

signature = inspect.signature(a)
signature.parameters.keys()
not any("RunContext" in str(signature.parameters[t].annotation) for t in signature.parameters.keys())

True

In [4]:
agent = AgentGENius(name="rot13encoder", model="openai:gpt-4o-mini", system_prompt="you write everything in rot13")

# add tool to toolset manually
agent.toolset.add(encode_rot13)


# or use decorator
@agent.tool_plain
def ask_user_tool(question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


print(f"{agent.toolset=}")
tools = agent.toolset.to_json()
print(f"{tools=}")
agent.toolset = ToolSet.from_json(tools, namespace=globals())
print(f"{agent.toolset=}")

result = agent.run_sync("tell me a haiku about ciphers")
print(result.data, "\n")
print(encode_rot13(result.data), "\n")

agent.toolset=ToolSet(['ask_agent', 'encode_rot13'])
tools='{"toolset":["ask_agent","encode_rot13"]}'
agent.toolset=ToolSet(['encode_rot13'])
Pvcuref jrnir gurve pbqr,  
Frpergf uvqqra va gur yvtug,  
Jbeqf qnapr va funqbjf. 

Ciphers weave their code,  
Secrets hidden in the light,  
Words dance in shadows. 



## Testing ToolSet

In [5]:
def test_tool(value: int) -> str:
    return str(value)


tools = ToolSet(test_tool)

tool = tools.get("test_tool")
assert tool(10) == "10"

assert "test_tool2" not in tools
assert "test_tool" in tools
assert test_tool in tools

tools.remove("test_tool")
tool = tools.get("test_tool")
assert tool is None

## Testing Serialization

In [6]:
def get_datetime(format: str = "%Y-%m-%d %H:%M:%S") -> str:
    """returns the current datetime as a string in the specified python format"""
    import datetime

    return datetime.datetime.now().strftime(format)


tools = ToolSet(get_datetime)

agent = AgentGENius(
    name="assistant", model="openai:gpt-4o-mini", system_prompt="You are a helpful assistant.", toolset=tools
)


# the date will be injected into the system prompt, but only for the first agent
# so it will not use a tool
@agent.system_prompt
def inject_date() -> str:
    return f"The date is {get_datetime()}."


print(f"{agent.to_json()=}")

# we cannot inject the date into the second agents system prompt, so it will use a tool
j = agent.to_json()
agent2 = BaseAgent.from_json(j, globals())


print(f"{agent2.to_json()=}")

print(agent.run_sync("What is the current date?").data)
print(agent2.run_sync("What is the current date?").data)

agent.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":["get_datetime","ask_agent"]}'
agent2.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":["get_datetime","ask_agent"]}'
The current date is 2025-01-03 08:32:12.
The current date is January 3, 2025.


In [1]:
from agentgenius.builtin_tools import *

# print(f"{get_builtin_tools_tool()=}")
print(f"{get_all_tools_tool()=}")
print(f"{get_all_agents_tool()=}")

get_all_tools_tool()=['get_all_agents_tool', 'get_all_tools_tool', 'get_datetime_tool', 'get_ram_stats', 'rot13_encode', 'get_username', 'get_user_ip']
get_all_agents_tool()=['ROT13_encoder']
